In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import GridSearchCV

In [ ]:
road = 7
direction = "out"
month = "03"
df = pd.read_csv(
    f"../dataset/2020-{month}-road-{road}-{direction}.csv", parse_dates=["datetime"])
df


In [ ]:
df_km127 = pd.read_csv("../dataset/latlon_km127.csv")

In [ ]:
import joblib
dtree_model = joblib.load(f'../Models/DecisionTreeModelRoad7in.joblib')
dtree_model

In [ ]:
def map_traffic_with_latlon(df):
    df['lat'] = df.apply(lambda row: df_km127[(df_km127['rd'] == row['road_number']) & (df_km127['km'] == row['km'])]['lat'].values[0]
                         if len(df_km127[(df_km127['rd'] == row['road_number']) & (df_km127['km'] == row['km'])]['lat'].values) > 0 else 0, axis=1)
    df['lon'] = df.apply(lambda row: df_km127[(df_km127['rd'] == row['road_number']) & (df_km127['km'] == row['km'])]['lon'].values[0]
                         if len(df_km127[(df_km127['rd'] == row['road_number']) & (df_km127['km'] == row['km'])]['lon'].values) > 0 else 0, axis=1)
    return df


In [ ]:
header_list = ['datetime', 'road_number', 'km', 'direction', 'all_units', 'inflow_units',
               'outflow_unit', 'samecell_units', 'avg_speed', 'max_speed', 'avg_traveltime', 'max_traveltime']
df_test = pd.read_csv("../dataset/current_celldata.csv",
                         names=header_list, parse_dates=["datetime"])
df_test = df_test[(df_test['road_number'] == 1) | (
    df_test['road_number'] == 2) | (df_test['road_number'] == 7)]
df_test = map_traffic_with_latlon(df_test)
df_test


In [ ]:
# num_cols = ['all_units', 'inflow_units',
#             'avg_speed', 'max_speed',
#             'avg_traveltime', 'max_traveltime']
# num_cols


In [ ]:
num_cols = ['all_units', 'inflow_units',
            'avg_speed', 'max_speed',
            'max_traveltime']
num_cols


In [ ]:
x_predict = df[num_cols]
x_predict

In [ ]:
y_predict = dtree_model.predict(x_predict)
y_predict

In [ ]:
df["cluster"] = y_predict
df

In [ ]:
df[df["cluster"] == 1].max(axis=0)

In [ ]:
df_acc = df[((df["cluster"] == 0) | (df["cluster"] == 1)) & (df["datetime"] > pd.Timestamp(2020, 3, 13, 8, 40))
                  & (df["datetime"] < pd.Timestamp(2020, 3, 13, 8, 59 ))]
df_acc


In [ ]:
# df_acc[(df_acc["cluster"]==1) & (df_acc["all_units"]>0) & (df_acc["km"]==1)]
df_acc[(df_acc["km"]==14)]